In [ ]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [ ]:
def combine(input, output_path):
    # combine all results
    df = pd.DataFrame()
    for search_result in input:
        df_single = pd.read_csv(search_result, header=None, sep = ",")
        # df = df.append(df_single, ignore_index=True, sort=False)
        df = pd.concat([df, df_single], ignore_index=True, sort=False)
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    df.to_csv(output_path, header=False, index=False)
# --------------------start of test code--------------------
# gos = fpath.poten_litera_gs_processed_step2
# wos = fpath.poten_litera_wos_processed
# pubmed = fpath.poten_litera_pubmed_processed
# eupmc = fpath.poten_litera_eupmc_processed
# input = [gos, wos, pubmed, eupmc]
# output_path = fpath.poten_litera_combined
# # plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# combine(input, output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# print(df.shape)
# # (14627, 8)
# ---------------------end of test code---------------------

In [ ]:
# fill in elements missed, remove duplciations based on identifiers in the potential related literature
def fill_in_missing(input_path, output_path, start, end):
    df = pd.read_csv(input_path, header=None, sep = ",")
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    
    # fill in elements that are missing
    for ind in range(start, end):
        # initialzie
        doi = np.nan
        pmid = np.nan
        pmcid = np.nan
        title = str(df["Title"][ind]).strip()
        # print(title)
        full_text_link = np.nan
        pdf_link = np.nan

        # doi, pmid
        if df["DOI"][ind] == df["DOI"][ind]: # DOI -> PMID
            doi = str(df["DOI"][ind]).strip()
            # print(doi)
            if df["PMID"][ind] == df["PMID"][ind]:
                pmid = str(df["PMID"][ind]).strip()
                # print(pmid)
            else:
                pmid = plib.doi2pmid(doi)
                # print(pmid)
                if pmid != pmid:
                    pmid_cadidate = plib.title2pmid(title)
                    if pmid_cadidate == pmid_cadidate:   
                        doi_validate, a = plib.pmid2doi_pmcid(pmid_cadidate)
                        if doi_validate == doi:
                            pmid = pmid_cadidate
                            # print(pmid)
        elif df["PMID"][ind] == df["PMID"][ind]: # PMID -> DOI
            pmid = str(df["PMID"][ind]).strip()
            # print(pmid)
            doi, pmcid = plib.pmid2doi_pmcid(str(pmid).strip())
            # print(doi)
        elif df["PMCID"][ind] == df["PMCID"][ind]: # PMCID -> DOI, PMID
            pmcid = str(df["PMCID"][ind]).strip()
            doi, pmid = plib.pmcid2doi_pmid(pmcid)
            # print(doi)
            # print(pmid)
        else:
            doi = np.nan
            pmid = np.nan
        # print(doi)
        # print(pmid)
        
        # pmcid
        if df["PMCID"][ind] == df["PMCID"][ind]:
            pmcid = str(df["PMCID"][ind]).strip()
        elif pmid == pmid:
            doi_1, pmcid = plib.pmid2doi_pmcid(str(pmid).strip())
        else:
            pmcid = np.nan
        # print(pmcid)

        title.lower()

        # full_text_link
        if pmcid == pmcid:
            full_text_link = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
        elif doi == doi:
            full_text_link = plib.get_final_redirected_url(str("https://doi.org/" + doi).strip())
        elif df["full_text_url"][ind] == df["full_text_url"][ind]:
            full_text_link = plib.get_final_redirected_url(df["full_text_url"][ind])
        else:
            full_text_link = np.nan
        # print(full_text_link)

        # pdf_link
        if df["pdf_url"][ind] == df["pdf_url"][ind]:
            pdf_link = plib.get_final_redirected_url(str(df["pdf_url"][ind]).strip())
        else:
            pdf_link = np.nan
        # print(pdf_link)
    
        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [title],
            "full_text_link": [full_text_link],
            "pdf_link": [pdf_link],
        }

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")

        print(ind)
# --------------------start of test code--------------------
input_path = fpath.poten_litera_combined
output_path = fpath.poten_litera_filled
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
fill_in_missing(input_path, output_path, 0, 14627)
# fill_in_identifiers(input, output, start, end)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def merge_remove_dupli(input_path, output_path): 
    df = pd.read_csv(input_path, header=None, sep = ",")

    # remove all duplicates
    df = df.drop_duplicates(subset=['DOI'])
    df = df.drop_duplicates(subset=['PMID'])
    df = df.drop_duplicates(subset=['PMCID'])
    print("Duplication in the potential related literature removed.")

    print("Found " + len(df) + " potential related literature in total.")

    # write the results into our output file
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
    df.to_csv(output_path, header=True, index=False)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_filled
# output_path = fpath.filtered_poten_litra
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
# print(df.head(3))
# print(df.shape)
# # 
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # merge all search results
# identifier = ["DOI", "PMID", "PMCID"]
# merge_remove_dupli(input_path, output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
# www.nature.com
def www_nature_com(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    try:
        elems = driver.find_elements(By.XPATH, "//span[@class='c-bibliographic-information__value')]")
        for elem in elems:
            if "doi.org/" in elem.text:
                doi = elem.text.split("doi.org/")[1]
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan
    try:
        title = driver.find_element(By.XPATH, "//h1[contains(@class, 'c-article-title')]").text
    except:
        title = np.nan
    abstract = np.nan
    keywords = np.nan
    intro = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
# url = "https://www.nature.com/articles/387281a0"
# info = www_nature_com(url)
# print(info["doi"])
# print(info["pmid"])
# print(info["pmcid"])
# print(info["title"])
# print(info["abstract"])
# print(info["keywords"])
# print(info["introduction"])
# print(info["pdf_link"])
# ---------------------end of test code---------------------